# Job Placement Dataset

In [734]:
# import packages
import matplotlib as plt
import numpy as np
import pandas as pd
import plotly.express as px
import random
import researchpy as rp
import scipy.stats as stats
import sklearn
from sklearn import compose
from sklearn.compose import make_column_selector
from sklearn.compose import make_column_transformer
from sklearn import discriminant_analysis
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn import neighbors
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

## Data Exploration

Data for this analysis were sourced from Kaggle (https://www.kaggle.com/datasets/ahsan81/job-placement-dataset). The objective of this analysis is to predict job placement status using various attributes of candidate's educational and professional history. The data dictionary supplied by the original poster is given below:

**Data Dictionary**  
gender : Gender of the candidate  
ssc_percentage : Senior secondary exams percentage (10th Grade)  
ssc_board : Board of education for ssc exams  
hsc_percentage : Higher secondary exams percentage (12th Grade)  
hsc_borad : Board of education for hsc exams  
hsc_subject : Subject of study for hsc  
degree_percentage : Percentage of marks in undergrad degree  
undergrad_degree : Undergrad degree majors  
work_experience : Past work experience  
emp_test_percentage : Aptitude test percentage  
specialization : Postgrad degree majors - (MBA specialization)  
mba_percent : Percentage of marks in MBA degree  
status (TARGET) : Status of placement. Placed / Not Placed

In [735]:
# read data from .csv file
data = pd.read_csv('Job_Placement_Data.csv')

# peek at data
display(data.head())

# check data set dimensions
display(data.shape)

,gender,ssc_percentage,ssc_board,hsc_percentage,hsc_board,hsc_subject,degree_percentage,undergrad_degree,work_experience,emp_test_percentage,specialisation,mba_percent,status
0,M,67.00,Others,91.00,Others,Commerce,58.00,Sci&Tech,No,55.0,Mkt&HR,58.80,Placed
1,M,79.33,Central,78.33,Others,Science,77.48,Sci&Tech,Yes,86.5,Mkt&Fin,66.28,Placed
2,M,65.00,Central,68.00,Central,Arts,64.00,Comm&Mgmt,No,75.0,Mkt&Fin,57.80,Placed
3,M,56.00,Central,52.00,Central,Science,52.00,Sci&Tech,No,66.0,Mkt&HR,59.43,Not Placed
4,M,85.80,Central,73.60,Central,Commerce,73.30,Comm&Mgmt,No,96.8,Mkt&Fin,55.50,Placed


(215, 13)

In [736]:
# check data types
display(data.info())

# check for missing values
display(data.isna().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 215 entries, 0 to 214
Data columns (total 13 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               215 non-null    object 
 1   ssc_percentage       215 non-null    float64
 2   ssc_board            215 non-null    object 
 3   hsc_percentage       215 non-null    float64
 4   hsc_board            215 non-null    object 
 5   hsc_subject          215 non-null    object 
 6   degree_percentage    215 non-null    float64
 7   undergrad_degree     215 non-null    object 
 8   work_experience      215 non-null    object 
 9   emp_test_percentage  215 non-null    float64
 10  specialisation       215 non-null    object 
 11  mba_percent          215 non-null    float64
 12  status               215 non-null    object 
dtypes: float64(5), object(8)
memory usage: 22.0+ KB


None

gender                 0
ssc_percentage         0
ssc_board              0
hsc_percentage         0
hsc_board              0
hsc_subject            0
degree_percentage      0
undergrad_degree       0
work_experience        0
emp_test_percentage    0
specialisation         0
mba_percent            0
status                 0
dtype: int64

In [737]:
# calculate job placement rate
display(data['status'].value_counts())
print(f"Placement Rate: {np.mean(data['status'] == 'Placed').round(3)}")

Placed        148
Not Placed     67
Name: status, dtype: int64

Placement Rate: 0.688


### Numeric Predictors

A brief analysis of the numeric predictors in the dataset using visualizations and likelihood ratio tests concluded that *ssc_percentage*, *hsc_percentage*, and *degree_percentage* should likely be included in the final model. Additionally, there was some evidence to suggest that *emp_test_percentage* is associated with placement status so it will be included in future models as well

In [738]:
# summarize numeric predictors
data.describe()

,ssc_percentage,hsc_percentage,degree_percentage,emp_test_percentage,mba_percent
count,215.000000,215.000000,215.000000,215.000000,215.000000
mean,67.303395,66.333163,66.370186,72.100558,62.278186
std,10.827205,10.897509,7.358743,13.275956,5.833385
min,40.890000,37.000000,50.000000,50.000000,51.210000
25%,60.600000,60.900000,61.000000,60.000000,57.945000
50%,67.000000,65.000000,66.000000,71.000000,62.000000
75%,75.700000,73.000000,72.000000,83.500000,66.255000
max,89.400000,97.700000,91.000000,98.000000,77.890000


In [739]:
# likelihood ratio test function from http://rnowling.github.io/machine/learning/2017/10/07/likelihood-ratio-test.html
def likelihood_ratio_test(features_alternate, labels, lr_model, features_null=None):
    """
    Compute the likelihood ratio test for a model trained on the set of features in
    `features_alternate` vs a null model.  If `features_null` is not defined, then
    the null model simply uses the intercept (class probabilities).  Note that
    `features_null` must be a subset of `features_alternative` -- it can not contain
    features that are not in `features_alternate`.
    Returns the p-value, which can be used to accept or reject the null hypothesis.
    """
    labels = np.array(labels)
    features_alternate = np.array(features_alternate)
    
    if features_null:
        features_null = np.array(features_null)
        
        if features_null.shape[1] >= features_alternate.shape[1]:
            raise (ValueError, "Alternate features must have more features than null features")
        
        lr_model.fit(features_null, labels)
        null_prob = lr_model.predict_proba(features_null)[:, 1]
        df = features_alternate.shape[1] - features_null.shape[1]
    else:
        null_prob = sum(labels) / float(labels.shape[0]) * \
                    np.ones(labels.shape)
        df = features_alternate.shape[1]
    
    lr_model.fit(features_alternate, labels)
    alt_prob = lr_model.predict_proba(features_alternate)

    alt_log_likelihood = -sk.metrics.log_loss(labels,
                                   alt_prob,
                                   normalize=False)
    null_log_likelihood = -sk.metrics.log_loss(labels,
                                    null_prob,
                                    normalize=False)

    G = 2 * (alt_log_likelihood - null_log_likelihood)
    p_value = stats.chi2.sf(G, df)

    return p_value

In [740]:
# boxplot factored by status
px.box(x = data['status'], 
       y = data['ssc_percentage'],
       labels = {
            'x': 'Placement Status',
            'y': 'SSC Percentage'
       },
       title = 'SSC Percentage Comparison').show()

# fit logistic regression model on single predictor
logFit = LogisticRegression().fit(np.array(data['ssc_percentage']).reshape(-1, 1), 
                                  data['status'])

# perform likelihood ratio test for independence
lrtPval = likelihood_ratio_test(np.array(data['ssc_percentage']).reshape(-1, 1), 
                                (data['status'] == 'Placed').astype(int), 
                                logFit)
print(f"LRT p-value: {lrtPval}")

LRT p-value: 8.939383353715161e-23


In [741]:
# boxplot factored by status
px.box(x = data['status'], 
       y = data['hsc_percentage'],
       labels = {
            'x': 'Placement Status',
            'y': 'HSC Percentage'
       },
       title = 'HSC Percentage Comparison').show()

# fit logistic regression model on single predictor
logFit = LogisticRegression().fit(np.array(data['hsc_percentage']).reshape(-1, 1), 
                                  data['status'])

# perform likelihood ratio test for independence
lrtPval = likelihood_ratio_test(np.array(data['hsc_percentage']).reshape(-1, 1), 
                                (data['status'] == 'Placed').astype(int), 
                                logFit)
print(f"LRT p-value: {lrtPval}")

LRT p-value: 2.2508006047868448e-15


In [742]:
# boxplot factored by status
px.box(x = data['status'], 
       y = data['degree_percentage'],
       labels = {
            'x': 'Placement Status',
            'y': 'Degree Percentage'
       },
       title = 'Degree Percentage Comparison').show()

# fit logistic regression model on single predictor
logFit = LogisticRegression().fit(np.array(data['degree_percentage']).reshape(-1, 1), 
                                  data['status'])

# perform likelihood ratio test for independence
lrtPval = likelihood_ratio_test(np.array(data['degree_percentage']).reshape(-1, 1), 
                                (data['status'] == 'Placed').astype(int), 
                                logFit)
print(f"LRT p-value: {lrtPval}")

LRT p-value: 2.044428524054884e-14


In [743]:
# boxplot factored by status
px.box(x = data['status'], 
       y = data['emp_test_percentage'],
       labels = {
            'x': 'Placement Status',
            'y': 'EMP Test Percentage'
       },
       title = 'EMP Test Percentage Comparison').show()

# fit logistic regression model on single predictor
logFit = LogisticRegression().fit(np.array(data['emp_test_percentage']).reshape(-1, 1), 
                                  data['status'])

# perform likelihood ratio test for independence
lrtPval = likelihood_ratio_test(np.array(data['emp_test_percentage']).reshape(-1, 1), 
                                (data['status'] == 'Placed').astype(int), 
                                logFit)
print(f"LRT p-value: {lrtPval}")

LRT p-value: 0.05918182852278074


In [744]:
# boxplot factored by status
px.box(x = data['status'], 
       y = data['mba_percent'],
       labels = {
            'x': 'Placement Status',
            'y': 'MBA Percentage'
       },
       title = 'MBA Percentage Comparison').show()

# fit logistic regression model on single predictor
logFit = LogisticRegression().fit(np.array(data['mba_percent']).reshape(-1, 1), 
                                  data['status'])

# perform likelihood ratio test for independence
lrtPval = likelihood_ratio_test(np.array(data['mba_percent']).reshape(-1, 1), 
                                (data['status'] == 'Placed').astype(int), 
                                logFit)
print(f"LRT p-value: {lrtPval}")

LRT p-value: 0.257167121513069


### Categorical Predictors

A brief analysis of the categorical predictors in the dataset using contingency tables and chi-squared independence tests concluded that *work_experience* and *specialisation* should likely be included in the final model. Additionally, there was some evidence to suggest that *gender* is associated with placement status so it will be included in future models as well since gender discrimination in hiring practices is a known and studied phenomenon. 

In [745]:
# sumamrize categorical predictors
findCatPredictors = make_column_selector(dtype_include = object)
rp.summary_cat(data[findCatPredictors(data)[0:(len(findCatPredictors(data)) - 1)]])

,Variable,Outcome,Count,Percent
0,gender,M,139,64.65
1,,F,76,35.35
2,ssc_board,Central,116,53.95
3,,Others,99,46.05
4,hsc_board,Others,131,60.93
5,,Central,84,39.07
6,hsc_subject,Commerce,113,52.56
7,,Science,91,42.33
8,,Arts,11,5.12
9,undergrad_degree,Comm&Mgmt,145,67.44


In [746]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['gender'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

gender,F,M
status,,
Not Placed,28,39
Placed,48,100


Chi-squared test p-value: 0.23980260881037568


In [747]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['ssc_board'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

ssc_board,Central,Others
status,,
Not Placed,38,29
Placed,78,70


Chi-squared test p-value: 0.6897729431530739


In [748]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['hsc_board'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

hsc_board,Central,Others
status,,
Not Placed,27,40
Placed,57,91


Chi-squared test p-value: 0.9222837052885


In [749]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['hsc_subject'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

hsc_subject,Arts,Commerce,Science
status,,,
Not Placed,5,34,28
Placed,6,79,63


Chi-squared test p-value: 0.5727118511343621


In [750]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['undergrad_degree'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

undergrad_degree,Comm&Mgmt,Others,Sci&Tech
status,,,
Not Placed,43,6,18
Placed,102,5,41


Chi-squared test p-value: 0.22661076190356658


In [751]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['work_experience'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

work_experience,No,Yes
status,,
Not Placed,57,10
Placed,84,64


Chi-squared test p-value: 9.906584720516494e-05


In [752]:
# construct contingency table
contingency = pd.crosstab(data['status'], data['specialisation'])
display(contingency)

# perform chi-squared test for independence
print(f"Chi-squared test p-value: {stats.chi2_contingency(contingency)[1]}")

specialisation,Mkt&Fin,Mkt&HR
status,,
Not Placed,25,42
Placed,95,53


Chi-squared test p-value: 0.00042018425858864284


## Model Fitting

In [753]:
# scale numeric predictors and encode categorical predictors
findNumPredictors = make_column_selector(dtype_include = float)
findCatPredictors = make_column_selector(dtype_include = object)
transform = make_column_transformer((StandardScaler(),
                                     findNumPredictors),
                                    (OneHotEncoder(drop = 'first'),
                                     findCatPredictors))

# get new column names
colNames = transform.fit(data.drop('status',
                                   axis = 1)).get_feature_names_out()

# transform data
modelDF = pd.DataFrame(transform.fit_transform(data.drop('status', 
                                                         axis = 1)),
                                                         columns = colNames)

np.random.seed(3172023) # set random seed

# split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(modelDF,
                                                    data['status'],
                                                    test_size = 0.2)

# rename predictor dataframes
X_train = pd.DataFrame(X_train,
                       columns = colNames)
X_test = pd.DataFrame(X_test,
                      columns = colNames)

# set of selected features
features = ['standardscaler__ssc_percentage',
            'standardscaler__hsc_percentage',
            'standardscaler__degree_percentage',
            'standardscaler__emp_test_percentage',
            'onehotencoder__gender_M',
            'onehotencoder__work_experience_Yes',
            'onehotencoder__specialisation_Mkt&HR']

A logistic regression model was fit using the selected predictors and achieved a cross-validation accuracy of 89%

In [754]:
np.random.seed(3172023) # set random seed

logFit = LogisticRegression()
cvResults = cross_val_score(logFit, 
                            X_train[features], 
                            y_train, 
                            cv = 10)

print(f"Cross-validation accuracy: {np.mean(cvResults).round(3)}")

Cross-validation accuracy: 0.89


A linear discriminant analysis model slightly underperformed the logistic regression model with a cross-validation accuracy of 88.4%.

In [755]:
np.random.seed(3172023) # set random seed

ldaFit = LinearDiscriminantAnalysis()
cvResults = cross_val_score(ldaFit, 
                            X_train[features], 
                            y_train, 
                            cv = 10)

print(f"Cross-validation accuracy: {np.mean(cvResults).round(3)}")

Cross-validation accuracy: 0.884


A K-Nearest Neighbors model performed still worse with a cross-validation accuracy of 86%.

In [756]:
np.random.seed(3172023) # set random seed

# tune KNN model
Kvalues = range(1, 20)
knnFit = KNeighborsClassifier()
knnTune = GridSearchCV(knnFit,
                       param_grid = {'n_neighbors': Kvalues},
                       cv = 10)
knnTune.fit(X_train[features],
            y_train)
cvResults = knnTune.cv_results_['mean_test_score']
K = Kvalues[np.where(cvResults == max(cvResults))[0][0]]

np.random.seed(3172023) # set random seed

# fit KNN model
knnFit = KNeighborsClassifier(n_neighbors = K)
cvResults = cross_val_score(knnFit,
                            X_train[features],
                            y_train,
                            cv = 10)

print(f"Cross-validation accuracy: {np.mean(cvResults).round(3)}")

Cross-validation accuracy: 0.86
